In [8]:
import numpy as np
import tensorflow as tf
import autograd
from autograd import numpy as anp

import scipy as sp
from scipy import optimize

from normal_example_lib import \
    normal_lp_suff, normal_lp, flatten_par, fold_par, get_objectives

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
dim = 3
num_obs = 10000

mu_true = np.arange(dim, dtype=np.float64)
sd_true = 0.5
tau_true = 1 / sd_true ** 2
print(tau_true)

x = np.random.normal(loc=mu_true, scale=sd_true, size=(num_obs, dim))
xsum = np.sum(x, axis=0)
x2sum = x.T @ x

muhat = xsum / num_obs
covhat = x2sum / num_obs - np.outer(muhat, muhat) 
tauhat = 1 / np.mean(np.diag(covhat))
print(tauhat)


4.0
3.9608268183883633


In [4]:
normal_lp_suff(
    mu=mu_true,
    mu2=np.outer(mu_true, mu_true),
    tau=tau_true,
    log_tau=np.log(tau_true),
    xsum=xsum,
    x2sum=x2sum,
    num_obs=num_obs)

<tf.Tensor: shape=(), dtype=float64, numpy=5644.917946321373>

In [5]:

data = { 'xsum': xsum, 'x2sum': x2sum, 'num_obs': num_obs}
par = { 'mu': mu_true, 'tau': tau_true }
normal_lp(par, data)

<tf.Tensor: shape=(), dtype=float64, numpy=5644.917946321373>

In [6]:
par_flat = flatten_par(par)
par_fold = fold_par(par_flat)
print(par['tau'], par_fold['tau'])
normal_lp(par_fold, data)

4.0 tf.Tensor(4.0, shape=(), dtype=float64)


<tf.Tensor: shape=(), dtype=float64, numpy=5644.917946321373>

In [9]:
normal_objective, normal_objective_grad, normal_objective_hessian = get_objectives(data)

lp = normal_objective(par_flat)
grad = normal_objective_grad(par_flat)
hess = normal_objective_hessian(par_flat)

lp, grad, hess

-0.5644917946321373
-0.5644917946321373
-0.5644917946321373


(-0.5644917946321373,
 array([0.0041134 , 0.02635507, 0.01430508, 0.01494975]),
 array([[4.        , 0.        , 0.        , 0.0041134 ],
        [0.        , 4.        , 0.        , 0.02635507],
        [0.        , 0.        , 4.        , 0.01430508],
        [0.0041134 , 0.02635507, 0.01430508, 1.51494975]]))

In [10]:
par_flat = np.random.random(dim  + 1)
lp = normal_objective(tf.convert_to_tensor(par_flat))
grad = normal_objective_grad(tf.convert_to_tensor(par_flat))
hess = normal_objective_hessian(par_flat)

lp, grad, hess

1.4743809018098275
1.4743809018098275
1.4743809018098275


(1.4743809018098275,
 array([ 0.22503288, -1.63814283, -2.10765742,  1.03638866]),
 array([[ 2.02993784,  0.        ,  0.        ,  0.22503288],
        [ 0.        ,  2.02993784,  0.        , -1.63814283],
        [ 0.        ,  0.        ,  2.02993784, -2.10765742],
        [ 0.22503288, -1.63814283, -2.10765742,  2.53638866]]))

In [11]:
opt_result = sp.optimize.minimize(
    x0=np.zeros(dim + 1),
    fun=normal_objective,
    jac=normal_objective_grad,
    hess=normal_objective_hessian,
    method="bfgs")

2.8649961274781663
2.8649961274781663
1.5671220266823649
1.5671220266823649
0.25571791977009994
0.25571791977009994
22.97887964309611
22.97887964309611
-0.07364546753740879
-0.07364546753740879
-0.30879477153213464
-0.30879477153213464
-0.5563902260978016
-0.5563902260978016
-0.5554561204210333
-0.5554561204210333
-0.5645117470138377
-0.5645117470138377
-0.5646160984482785
-0.5646160984482785
-0.564679189708661
-0.564679189708661
-0.564679193991477
-0.564679193991477


/home/rgiordan/.local/lib/python3.6/site-packages/scipy/optimize/_minimize.py:511: RuntimeWarning: Method bfgs does not use Hessian information (hess).
  RuntimeWarning)


In [12]:
lp = normal_objective(opt_result.x)
grad = normal_objective_grad(opt_result.x)
hess = normal_objective_hessian(opt_result.x)
grad, hess

-0.564679193991477
-0.564679193991477
-0.564679193991477


(array([ 7.49257057e-10, -7.23800929e-07, -1.45459728e-06,  4.12283555e-07]),
 array([[ 3.96082791e+00,  0.00000000e+00,  0.00000000e+00,
          7.49257056e-10],
        [ 0.00000000e+00,  3.96082791e+00,  0.00000000e+00,
         -7.23800929e-07],
        [ 0.00000000e+00,  0.00000000e+00,  3.96082791e+00,
         -1.45459728e-06],
        [ 7.49257057e-10, -7.23800929e-07, -1.45459728e-06,
          1.50000041e+00]]))

In [ ]:
par_opt = fold_par(opt_result.x)
print(par_opt['mu'].numpy(), muhat)
print(par_opt['tau'].numpy(), tauhat, tau_true)